In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

plt.style.use("fivethirtyeight")
sns.set_style("darkgrid")

In [ ]:

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc, roc_curve, roc_auc_score

from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance

from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder# Imports from our package


In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv")

In [ ]:
display(train.head())
display(test.head())
display(sub.head())

In [ ]:
trainoriginal = train.copy()
testoriginal = test.copy()

In [ ]:
print("train:", train.shape)
print("test:", test.shape)
print("Submission:", sub.shape)

In [ ]:
print('train_columns:',train.columns)
print('test_columns:',test.columns)

In [ ]:
print('train_info:',train.info())
print('test_info:',test.info())

In [ ]:
#不要なidを除去
train=train.drop(['id'],axis=1)

test=test.drop(['id'],axis=1)

In [ ]:
#欠損値の確認
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['target'].value_counts().sort_index(ascending=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='target',data=train)

In [ ]:
#targetの中身を数値として振り分ける
#LabelEncoderとほぼ同じやりとり
train['target1']=train['target'].map({'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3})

In [ ]:
train

In [ ]:
for i in train.columns:
    print(train[i].value_counts())

In [ ]:
plt.figure(figsize=(20,30))
sns.boxplot(data=train,orient='h')

In [ ]:
#target1との相関係数
train.corr()['target1'].sort_values(ascending=False)

In [ ]:
a = train.drop(['target','target1'], axis=1)
a.corrwith(train['target1']).plot(kind='bar', figsize=(18,11), color=['red'])
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()

In [ ]:
#歪度→分布の非対称性を表す指標｡値が正であれば､分布が右に伸びていることになる｡値が負だと左に伸びていることになる｡
train.skew()

In [ ]:
test.skew()

In [ ]:
#列は全部指定して､行はtargetとtarget1を除く
#x→説明変数
X=train.iloc[:,:-2]
#y→目的変数
y=train['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
print('X_train:{i}'.format(i=len(X_train)))
print('X_test:{i}'.format(i=len(X_test)))
print('y_train:{i}'.format(i=len(y_train)))
print('y_test:{i}'.format(i=len(y_test)))

In [ ]:
#XGBoost
model=XGBClassifier(ranoam_state=42,use_label_encoder=True)
model.fit(X,y)

In [ ]:
#predict()とpredict_proba()
y_pred_xgb=model.predict_proba(test)
print(y_pred_xgb)

In [ ]:
submission_xgb=pd.DataFrame(y_pred_xgb,columns=['Class_1','Class_2','Class_3','Class_4'])
submission_xgb['id']=sub['id']
submission_xgb

In [ ]:
LGB = LGBMClassifier(random_state=42, use_label_encoder=True)
LGB.fit(X, y)

In [ ]:
plot_importance(LGB, figsize=(18, 15));

In [ ]:
n_threads=4
n_folds=5
random_state=10
test_size=0.2
timeout=60*60
le=LabelEncoder()
train['target']=le.fit_transform(train['target'])

In [ ]:
pip install -U lightautoml

In [ ]:
# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from sklearn.metrics import log_loss

In [ ]:
automl=TabularUtilizedAutoML(task=Task('multiclass'),
                           timeout=timeout,
                           cpu_limit=n_threads,
                           verbose=100,
                           reader_params={'n_jobs':n_threads},
                           )

In [ ]:
target_columns='target'
roles={
    'target':target_columns,
    'drop':['id'],
}
#trainのtarget1を以外
lightml_pred=automl.fit_predict(train.iloc[:,:-1],roles=roles)
print(lightml_pred)
print(lightml_pred.shape)

In [ ]:
test_pred = automl.predict(test)
sub.iloc[:, 1:] = test_pred.data
sub.to_csv('Submission1.csv', index = False)